<a href="https://colab.research.google.com/github/jaime-villela/python-tips/blob/main/Classes_In_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Classes In Python
For this article, I'm not going to give an example of how I use classes in Python to make my code more readable.  I won't go into all the aspects of Object Oriented Programming (OOP) becase that's a much broader topic.  Rather, I want to show the usefulness of two principles of OOP, namely *Encapsulation* and *Abstraction*.  I will attempt to demonstrate these two principles by making improvements on the code I wrote to implement the Nelder-Mead method in this [notebook I previously created.](https://colab.research.google.com/drive/1nIcvS_0CyuGm3VqkauLeYZRJQJF9ZaVW?usp=sharing)
<br>
<br>
**Encapsulation** refers to the idea that only some of the data you are working with needs to be exposed to the world.  For example, the algorithm I implemented relies on a matrix of vertices which make up a triangle.  As algorithm progresses, the shape of the triangle is modified.  In the process of transforming the shape of the triangle, new points need to be calculated for reflection, contraction, expansion, etc.  Those intermediate points don't need to be made pulic as they are only relevant momentarily.  Therefore, it makes sense to keep those bits of data encapsulated within an object.
<br>
<br>
**Abstraction** refers to the idea that all code needs to be made public.  In keeping with the example of the shape-shifting triangle, the points for reflection, contraction, etc, need to be calculated but those calculations don't need to be made known to the world.  A convenient side-effect of this principle is that you can wrap several statements within a function and then give that function a name that describes what is going on "behind the scenes".


## First, the pseudocode
We'll start by showing the Nelder-Mead algorithm written here in pseudocode.

```
IF f (R) < f (G) THEN 
    Perform Case (i) {either reflect or extend}
ELSE 
    Perform Case (ii) {either contract or shrink}

BEGIN {Case (i).} # reflect or extend
    IF f (B) < f (R) THEN 
        replace W with R 
    ELSE
        Compute E and f (E) 
        IF f (E) < f (B) THEN 
            replace W with E 
        ELSE 
            replace W with R
        ENDIF 
    ENDIF 
END {Case (i).}


BEGIN {Case (ii).} # contract or shrink
    IF f (R) < f (W) THEN
        replace W with R
     
    Compute C = (W + M)/2 or C = (M + R)/2 and f (C)
    IF f (C) < f (W) THEN
        replace W with C
    ELSE
        Compute S and f (S)
        replace W with S
        replace G with M
    ENDIF
END {Case (ii).}
```

If we compare the pseudocode to the actual code (shown below) you see that the actual code has a lot more steps in it.  That's because each step implies some calculations that have to take place.  Notice also that there are lots of arrays being indexed.  Even though the names of the rows and columns are visible, it can become hard to follow what's going on.  I also try to always use long variable names that express what the variable holds but imagine if I had decided to only use single letters or acronmys instead.  
<br>
Wouldn't it be nice if the actual code read as simply as the pseudocode?  That is, after all, why pseudocode exists, because it's easier to understand complete sentences than language syntax.

## Now, The Actual Code

In [ ]:
while(did_points_converge(vertices['z']) == False):
    Midpoint = (vertices.loc['Best'] + vertices.loc['Good']) / 2
    Reflection = 2 * Midpoint - vertices.loc['Worst']
    Reflection['z'] = func(Reflection['x'], Reflection['y'])

    if Reflection['z'] < vertices.loc['Good']['z']:
        print('Case 1 >>>>')
        if vertices.loc['Best']['z'] < Reflection['z']:
            print('Replace W with R')
            vertices.loc['Worst'] = Reflection
        else:
            Extension = 2 * Reflection - Midpoint
            Extension['z'] = func(Extension['x'], Extension['y'])
            
            if Extension['z'] < vertices.loc['Best']['z']:
                print('Replace W with E')
                vertices.loc['Worst'] = Extension
            else:
                print('Replace W with R')
                vertices.loc['Worst'] = Reflection
    else:
        print('Case 2 >>>>')
        if Reflection['z'] < vertices.loc['Worst']['z']:
            print('Replace W with R')
            vertices.loc['Worst'] = Reflection
        else:
            C1 = (vertices.loc['Worst'] + Midpoint) / 2
            C2 = (Midpoint + Reflection) / 2
            C1['z'] = func(C1['x'], C1['y'])
            C2['z'] = func(C2['x'], C2['y'])
            Contraction = C1 if C1['z'] < C2['z'] else C2

            if Contraction['z'] < vertices.loc['Worst']['z']:
                print('Replace W with C')
                vertices.loc['Worst'] = Contraction
            else:
                Shrinkpoint = (vertices.loc['Best'] + vertices.loc['Worst']) / 2
                Shrinkpoint['z'] = func(Shrinkpoint['x'], Shrinkpoint['y'])
                print('Replace W with S')
                print('Replace G with M')
                vertices.loc['Worst'] = Shrinkpoint
                vertices.loc['Good'] = Midpoint

    vertices = sort_and_rename_vertices(vertices)
    print(vertices)

# Classes To The Rescue
Here's where a class can help make our code simpler to read (and, thus, maintain).  If we create a class for our simplex triangle, the details of each step in the algorithm no longer need to be explicitly stated.  Instead, we can write our class methods (functions) so that they describe exactly what's happening.

## The Simplex Class Triangle
Before looking at the class implementation, I would recommend having a look at the modified code in the next section.  Notice how the modified code looks a LOT like the pseudocode.  Even without looking at the class code, you should be able to look at the modified code and get a good understanding what is going on.  That's the power of classes.  Now, let's discuss the class.
<br>
<br>
As stated before, the algorithm revolves around a triangle whose vertices are manipulated.  So it makes sense to start by creating a class that will hold all of the information we need for each transformation.
<br>
<br>
The first thing to notice is that the constructor method (`__init__`) takes as parameters two arrays, one for the $x$ values and for the $y$ values.  Then, for each $(x, y)$ combination, it evaluates the fuction (also a parameter) and stores the results in the $z$ column.
<br>
<br>
The next thing you'll notice is that there are lots of helper functions.  Because the helper functions don't need to be made public, they are named with a starting underscore (`_`).  There are helper functions to calculate all of the points that can be required during the algorithm: reflection, contraction, extension, etc.  There also functions to hide the logical comparisons of the $z$ values.  I do this becuase it's easier to read a function call in an `if` statement than it is to read the actual logical expression.
<br>
<br>
For example, if I were to read the following statement out loud:
$$if\text{ }f(B) < f(R)$$
it would sound something like this, "if f-of-B is less-than f-of-R".  Therefore, I created a helper function to match this logic and I named it `is_f_of_B_lt_f_of_R`.  You get the idea, hopefully.  I start these functions with the word `is` because I want them to read like a question with a Yes or No answer.
<br>
<br>
The functions that are public do not begin with and underscore and have descriptive names.  For example, take a look at the `calc_contraction_point()` function on lines 69 to 74.  The name of the function tells you what is happening without you having to figure out what all is going on here
```
C1 = (self._vertices.loc['Worst'] + self._mid_B_to_G) / 2
C2 = (self._mid_B_to_G + self._Reflection) / 2
C1['z'] = func(C1['x'], C1['y'])
C2['z'] = func(C2['x'], C2['y'])
self._Contraction = C1 if C1['z'] < C2['z'] else C2
```
Plus, you just replaced five lines of code with one.
<br>
<br>
One final note about my naming convention, just as with the functions that don't need to be public, I named the private data members with a starting underscore (e.g. `_Reflection`).
Everything I just described is an example of **Encapsulation** and **Abstraction**.

In [ ]:
from numpy import float64
import pandas as pd

class SimplexTriangle:
    def __init__(self, x_array, y_array, func):
        # Note that the z values default to zero.
        # Also vertices is prepended with a single underscore to imply private.
        self._vertices = pd.DataFrame({'x': x_array, 
                                      'y': y_array, 
                                      'z': [0.0, 0.0, 0.0]})
        self._vertices.index = ['Best', 'Good', 'Worst']
        self.func = func
        # Now we can begin to fill in the z values.
        self._vertices.at['Best','z'] = func(self._vertices.at['Best', 'x'], 
                                            self._vertices.at['Best', 'y'])
        self._vertices.at['Good','z'] = func(self._vertices.at['Good', 'x'], 
                                            self._vertices.at['Good', 'y'])
        self._vertices.at['Worst','z'] = func(self._vertices.at['Worst', 'x'], 
                                            self._vertices.at['Worst', 'y'])
        # The Best vertex will have the least value of z so we reorder.
        self.sort_and_rename_vertices()
        self.print_vertices()
        self._mid_B_to_G = []
        self._Reflection = []
        self._Contraction = []
        self._Extension = []

    def sort_and_rename_vertices(self):
        self._vertices = self._vertices.sort_values(by='z')
        self._vertices.index = ['Best', 'Good', 'Worst']

    def print_vertices(self):
        print(self._vertices)

    def _calc_mid_B_to_G(self):
        self._mid_B_to_G = (self._vertices.loc['Best'] + 
                           self._vertices.loc['Good']) / 2

    def calc_reflection_point(self):
        self._calc_mid_B_to_G()
        self._Reflection = 2 * self._mid_B_to_G - self._vertices.loc['Worst']
        self._Reflection['z'] = func(self._Reflection['x'], self._Reflection['y'])

    def get_Reflection(self):
        return self._Reflection

    def is_f_of_R_lt_f_of_G(self):
        return True if self._Reflection['z'] < self._vertices.at['Good', 'z'] else False

    def is_f_of_B_lt_f_of_R(self):
        return True if self._vertices.at['Best','z'] < self._Reflection['z'] else False

    def replace_W_with_R(self):
        self._vertices.loc['Worst'] = self._Reflection

    def calc_extension_point(self):
        self._Extension = 2 * self._Reflection - self._mid_B_to_G
        self._Extension['z'] = func(self._Extension['x'], self._Extension['y'])

    def is_f_of_E_lt_f_of_B(self):
        return True if self._Extension['z'] < self._vertices.at['Best','z'] else False

    def replace_W_with_E(self):
        self._vertices.loc['Worst'] = self._Extension
        
    def is_f_of_R_lt_f_of_W(self):
        return True if self._Reflection['z'] < self._vertices.at['Worst', 'z'] else False

    def calc_contraction_point(self):
        C1 = (self._vertices.loc['Worst'] + self._mid_B_to_G) / 2
        C2 = (self._mid_B_to_G + self._Reflection) / 2
        C1['z'] = func(C1['x'], C1['y'])
        C2['z'] = func(C2['x'], C2['y'])
        self._Contraction = C1 if C1['z'] < C2['z'] else C2

    def is_f_of_C_lt_f_of_W(self):
        return True if self._Contraction['z'] < self._vertices.at['Worst', 'z'] else False

    def replace_W_with_C(self):
        self._vertices.loc['Worst'] = self._Contraction

    def calc_shrink_point(self):
        self._Shrinkpoint = (self._vertices.loc['Best'] + 
                             self._vertices.loc['Worst']) / 2
        self._Shrinkpoint['z'] = func(self._Shrinkpoint['x'], self._Shrinkpoint['y'])

    def replace_W_with_S(self):
        self._vertices.loc['Worst'] = self._Shrinkpoint

    def replace_G_with_M(self):
        self._vertices.loc['Good'] = self._mid_B_to_G

    def _is_a_close_to_b(self, a, b, tolerance=0.0001):
        return True if abs(a - b) < tolerance else False

    def are_points_converged(self):
        best_z = self._vertices.at['Best', 'z']
        good_z = self._vertices.at['Good', 'z']
        worst_z = self._vertices.at['Worst', 'z']

        if (self._is_a_close_to_b(best_z, good_z) and
            self._is_a_close_to_b(good_z, worst_z) and
            self._is_a_close_to_b(worst_z, best_z)):
            return True
        else:
            return False

def func(x,y):
    return x**2 - 4*x + y**2 - y - (x*y)

## The improved code
If we re-write the code using the class methods it will like the next block.  Notice how much the actual code resembles the pseudocode.  The function names reflect what is really going on in each step.  Even the conditions in each `if` statement describe what is being tested if you read the function name from left to right.

In [ ]:
x_array = [0.0, 1.2, 0.0]
y_array = [0, 0, 0.8]
triangle = SimplexTriangle(x_array, y_array, func)

while(triangle.are_points_converged() == False):
    triangle.calc_reflection_point()

    if triangle.is_f_of_R_lt_f_of_G():
        print('Case 1 >>>>')
        if triangle.is_f_of_B_lt_f_of_R():
            triangle.replace_W_with_R()
        else:
            triangle.calc_extension_point()
            
            if triangle.is_f_of_E_lt_f_of_B():
                triangle.replace_W_with_E()
            else:
                triangle.replace_W_with_R()
    else:
        print('Case 2 >>>>')
        if triangle.is_f_of_R_lt_f_of_W():
            triangle.replace_W_with_R()
        else:
            triangle.calc_contraction_point()
            
            if triangle.is_f_of_C_lt_f_of_W():
                triangle.replace_W_with_C()
            else:
                triangle.calc_shrink_point()
                triangle.replace_W_with_S()
                triangle.replace_G_with_M()
                
    triangle.sort_and_rename_vertices()
    triangle.print_vertices()

# Conclusion
Classes can make your code much easier to read--not to mention, easier to maintain.  Notice how much shorter and easier to understand the implementation is when using a class.  Yes, it requires more effort up front but the time you save after the fact (perhaps in debgugging?) will more than make up for the time it took to create all the extra code.